In [ ]:
path = '/kaggle/input'

In [ ]:
%pylab inline
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from pprint import pprint
from xgboost import XGBClassifier
from gensim.models import Phrases, LdaModel
from gensim.corpora import Dictionary
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import brown
from nltk import FreqDist
from wordcloud import WordCloud 
from collections import OrderedDict



In [ ]:
# Creating a Df with 5 columns: Directory, Category, FileName, Title, Text
categories = []
titles = []
all_data = []
#df = pd.DataFrame()
# dirname
for dirname, categoryname, filenames in os.walk(path):
    # filename
    for filename in filenames:
        if filename == 'README.TXT':
            filenames.remove(filename)
        else:
            # Absolute path
            current_file = os.path.abspath(os.path.join(dirname, filename))
            open_file = open(current_file, 'r', encoding="latin-1")
            # text_data
            text_data = open_file.read().split('\n')
            text_data = list(filter(None, text_data))
            titles.append(text_data[0])
            all_data.append((dirname, dirname.rsplit('/',1)[1], filename, text_data[0], text_data[1:]))
            #data_df = f"Directory: {dirname}, Category: {dirname.rsplit('/',1)[1]}, FileName: {filename}, Title: {text_data[0]}, Text: {text_data[1:]}"
            #print(data_df)
df = pd.DataFrame(all_data, columns=['directory', 'category', 'fileName', 'title', 'text'])
df['text'] = df.text.astype(str)
print(df.head())  

In [ ]:
df.describe()

In [ ]:
df.category.value_counts()

In [ ]:
bar_plot=df.category.value_counts().plot(kind='barh', figsize=(8, 6), color='teal')
plt.xlabel("Nr. of Artciles", labelpad=14)
plt.ylabel("Category", labelpad=14)
plt.title("Nr. of Articles in category", y=1.02, color='navy')

for index, value in enumerate(df.category.value_counts()):
    plt.text(value, index, str(value))

In [ ]:
df.text[1][:1000]

#### Label Encoder

In [ ]:
# 0 - business, 1 -entertainment, 2 - politics, 3 - sport, 4 - tech
label_enc = LabelEncoder()
df['label'] = label_enc.fit_transform(df['category'])
df.tail()

In [ ]:
# An array of words
df_txt = np.array(df['text'])

In [ ]:
df_txt

### Pre-process and vectorize text

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')

def docs_preprocessor(docs):
    # Remain only letters
    tokenizer = RegexpTokenizer('[A-Za-z]\w+')
    
    for idx in range(len(docs)):
         # Convert to lowercase
        docs[idx] = docs[idx].lower() 
        # Split into words
        docs[idx] = tokenizer.tokenize(docs[idx])  
    
    # Lemmatize all words with len>2 in documents 
    lemmatizer = WordNetLemmatizer()
    docs = [[nltk.stem.WordNetLemmatizer().lemmatize(token) for token in doc if len(token) > 2 and token not in stopwords] for doc in docs]
    #stemmer = SnowballStemmer('english')
    #docs = [[stemmer.stem(token) for token in doc if len(token) > 2 and token not in stopwords] for doc in docs]
         
    return docs

In [ ]:
df_txt = docs_preprocessor(df_txt)

In [ ]:
# Add bigrams and trigrams to docs (only ones that appear 10 times or more)
bigram = Phrases(df_txt, min_count=10)
trigram = Phrases(bigram[df_txt])

for idx in range(len(df_txt)):
    for token in bigram[df_txt[idx]]:
        if '_' in token:
            df_txt[idx].append(token)
    for token in trigram[df_txt[idx]]:
        if '_' in token:
            df_txt[idx].append(token)

Remove rare words:

In [ ]:
# Create a dictionary representation of the documents
dictionary = Dictionary(df_txt)
print('Nr. of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Nr. of unique words after removing rare and common words:', len(dictionary))

In [ ]:
df['text2'] = df_txt

In [ ]:
df['text3'] = [' '.join(map(str, j)) for j in df['text2']]

In [ ]:
df.iloc[1475:1480,:]

#### Word Vectors

In [ ]:
#vectorizer = TfidfVectorizer(stop_words = 'english', lowercase=True)
vectorizer = TfidfVectorizer(input='content', analyzer = 'word', lowercase=True, stop_words='english',\
                                   ngram_range=(1, 3), min_df=40, max_df=0.20,\
                                  norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=True)
text_vector = vectorizer.fit_transform(df.text3)
dtm = text_vector.toarray()
features = vectorizer.get_feature_names()

In [ ]:
h = pd.DataFrame(data = text_vector.todense(), columns = vectorizer.get_feature_names())
h.iloc[990:1000,280:300]

In [ ]:
corpus = [dictionary.doc2bow(txt) for txt in df_txt]

In [ ]:
print(f'Number of unique tokens: {len(dictionary)}')
print(f'Number of documents: {len(corpus)}')

In [ ]:
# Frequency distribution for dictionary
#fdist = nltk.FreqDist(dictionary)
#fdist

### Classification Model

In [ ]:
X = text_vector
y = df.label.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
svc1 = RandomForestClassifier(random_state = 42)
svc1.fit(X_train, y_train)
svc1_pred = svc1.predict(X_test)
#print(f"Train Accuracy: {svc1.score(X_train, y_train)*100:.3f}%")
print(f"Test Accuracy: {svc1.score(X_test, y_test)*100:.3f}%")

In [ ]:
svc2 = XGBClassifier(random_state = 42, use_label_encoder=False)
svc2.fit(X_train, y_train)
svc2_pred = svc2.predict(X_test)
#print(f"Train Accuracy: {svc2.score(X_train, y_train)*100:.3f}%")
print(f"Test Accuracy: {svc2.score(X_test, y_test)*100:.3f}%")

In [ ]:
svc3 = SGDClassifier(random_state = 42)
svc3.fit(X_train, y_train)
svc3_pred = svc3.predict(X_test)
#print(f"Train Accuracy: {svc3.score(X_train, y_train)*100:.3f}%")
print(f"Test Accuracy: {svc3.score(X_test, y_test)*100:.3f}%")

In [ ]:
svc4 = KNeighborsClassifier()
#pprint(svc4.get_params())
svc4.fit(X_train, y_train)
svc4_pred = svc4.predict(X_test)
#print(f"Train Accuracy: {svc4.score(X_train, y_train)*100:.3f}%")
print(f"Test Accuracy: {svc4.score(X_test, y_test)*100:.3f}%")

In [ ]:
# Function to calculate mean absolute error
def mae(y_true, y_pred):
    return np.mean(abs(y_true - y_pred))

# Takes in a model, trains the model, and evaluates the model on the test set
def fit_and_evaluate(model):
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions and evalute
    model_pred = model.predict(X_test)
    model_mae = mae(y_test, model_pred)
    
    # Return the performance metric
    return model_mae

In [ ]:
svc1_mae = fit_and_evaluate(svc1)
svc2_mae = fit_and_evaluate(svc2)
svc3_mae = fit_and_evaluate(svc3)
svc4_mae = fit_and_evaluate(svc4)
#print(svc1_mae, svc2_mae, svc3_mae, svc4_mae)

In [ ]:
plt.style.use('fivethirtyeight')
fig = plt.figure(figsize(8, 6))

# Dataframe to hold the results
model_comparison = pd.DataFrame({'model': ['RandomForest Classifier', 'XGBClassifier', 
                                           'SGDClassifier', 'KNeighborsClassifier'
                                          ],
                                 'mae': [svc1_mae, svc2_mae, 
                                         svc3_mae, svc4_mae]})

# Horizontal bar chart of test mae
model_comparison.sort_values('mae', ascending = False).plot(x = 'model', y = 'mae', kind = 'barh',
                                                           color = 'yellow', edgecolor = 'black')

# Plot formatting
plt.ylabel('')
plt.yticks(size = 14)
plt.xlabel('Mean Absolute Error')
plt.xticks(size = 14)
plt.title('Model Comparison on Test MAE', size = 20)

### Word Cloud

In [ ]:
wc = WordCloud(width = 800, height = 800, 
                background_color ='white', 
               stopwords=stopwords,
                min_font_size = 10, random_state=42).generate(df.text3.to_string())

plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wc)
plt.tight_layout(pad = 0) 
plt.axis("off")
plt.show()

In [ ]:
for x in df.category.unique():
    wc = WordCloud(width = 800, height = 800, background_color ='white', stopwords=stopwords,
                   min_font_size = 10, random_state=42)
    wc.generate(df.text3[(df.category == x)].to_string())
    
    plt.imshow(wc)
    plt.tight_layout(pad = 0) 
    plt.title(x)
    plt.axis("off")
    plt.show()

## Clustering

In [ ]:
# Set training parameters
num_topics = 5
chunksize = 500 # Number of documents to consider at once (affects the memory consumption)
passes = 20 # Number of passes through documents
iterations = 400
eval_every = 1  

# Make a index to word dictionary
temp = dictionary[0] 
id2word = dictionary.id2token

model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', random_state=78, \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

In [ ]:
# Frequency of top words in each topic
def explore_topic(lda_model, topic_number, topn, output=True):
    terms = []
    for term, frequency in lda_model.show_topic(topic_number, topn=topn):
        terms += [term]
        if output:
            print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))   
    return terms

topic_summaries = []

print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')
for i in range(num_topics):
    print('Topic '+str(i)+'\n')
    tmp = explore_topic(model,topic_number=i, topn=10, output=True)
    topic_summaries += [tmp[:5]]
    print('\n')

In [ ]:
# Assign a human-interpretable 
top_labels = {0: 'business', 1:'sport', 2:'tech', 3:'entertainment', 4:'politics'}

In [ ]:
top_dist =[]
for d in corpus:
    tmp = {i:0 for i in range(num_topics)}
    tmp.update(dict(model[d]))
    vals = list(OrderedDict(tmp).values())
    top_dist += [array(vals)]

def get_doc_topic_dist(model, corpus, kwords=False):
    '''
    LDA transformation, for each doc only returns topics with non-zero weight
    This function makes a matrix transformation of docs in the topic space.
    
    model: the LDA model
    corpus: the documents
    kwords: if True adds and returns the keys
    '''
    top_dist =[]
    keys = []

    for d in corpus:
        tmp = {i:0 for i in range(num_topics)}
        tmp.update(dict(model[d]))
        vals = list(OrderedDict(tmp).values())
        top_dist += [array(vals)]
        if kwords:
            keys += [array(vals).argmax()]

    return array(top_dist), keys

top_dist, lda_keys= get_doc_topic_dist(model, corpus, True)

In [ ]:
top_ws = []
for n in range(len(dtm)):
    inds = int0(argsort(dtm[n])[::-1][:4])
    tmp = [features[i] for i in inds]
    
    top_ws += [' '.join(tmp)]
    
df['Text_Rep'] = pd.DataFrame(top_ws)
df['clusters'] = pd.DataFrame(lda_keys)
df['clusters'].fillna(10, inplace=True)

cluster_colors = {0: 'blue', 1: 'green', 2: 'yellow', 3: 'red', 4: 'skyblue'}

df['colors'] = df['clusters'].apply(lambda j: cluster_colors[j])
# Assign a human-interpretable labels 
df['category_lda'] = df['clusters'].replace([0, 1, 2, 3, 4],['business','sport','tech','entertainment','politics'])

### Clusters visualization

In [ ]:
from bokeh.plotting import figure, show, output_notebook, save
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource

In [ ]:
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(top_dist)

df['X_tsne'] =X_tsne[:, 0]
df['Y_tsne'] =X_tsne[:, 1]

output_notebook()

source = ColumnDataSource(dict(
    x=df['X_tsne'],
    y=df['Y_tsne'],
    color=df['colors'],
    label=df['clusters'].apply(lambda l: top_labels[l]),
    topic_key= df['clusters'],
    title= df[u'title'],
    content = df['text3'],
    legend_field=df['category_lda']
))

df = df.drop(columns=['colors','Text_Rep','X_tsne','Y_tsne'])

In [ ]:
title = 'Topics visualization'

plot_lda = figure(plot_width=1000, plot_height=600,
                     title=title, 
                     x_axis_type=None, y_axis_type=None, min_border=1)
plot_lda.scatter(x='x', y='y', legend_field='legend_field',  source=source,
                 color='color', alpha=0.6, size=5.0)

# hover tools
hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips = {"content": "Title: @title, KeyWords: @content - Topic: @topic_key "}
plot_lda.legend.location = "top_left"

show(plot_lda)

In [ ]:
df.tail()

In [ ]:
print(f"Correct categories: {len(df[df.category==df.category_lda])}")

In [ ]:
print(f"Percentage of correct categories: {round(len(df[df.category==df.category_lda])/len(df)*100,2)}%")